## Возможности и ограничения языковых моделей

**Тут есть важное действие, которое нужно сделать перед тем, как запускать этот ноутбук**

Если вы делаете это задание в Google Colab, первым делом переключите Runtime в GPU. Это задание нормально посчитается и на CPU, но некоторые из будущих кейсов потребуют GPU (либо вам придётся несколько часов или даже дней, чтобы модель обучилась - и мы не преувиличиваем). Ещё мы рекомендуем переключить язык интерфейса на английский, потому что русская локализация ужасна да и вообще не нужна.

О том, как переключить рантайм: https://www.geeksforgeeks.org/how-to-use-google-colab/

In [1]:
!pip install datasets transformers[torch]==2.10.0 tqdm > /dev/null

In [2]:
from transformers import AutoModel, AutoTokenizer, GPT2LMHeadModel
import datasets

import torch
from torch.nn import functional as F
from tqdm import tqdm
import warnings

In [3]:
warnings.filterwarnings("ignore")

In [4]:
import transformers
print(transformers.__version__)  # should be above or equal 4.0.1

2.10.0


В этом модуле мы с вами познакомились с большими языковыми моделями и обсудили их возможности, а также ограничения. В этом задании вам предлагается поэкспериментировать с одной из больших языковых моделей -- GPT-2 -- и самим убедиться, так ли она хороша, как о ней рассказывают.

## Библиотека transformers

Вы уже сталкивались с библиотекой transformers в этом курсе.

Мы [загрузим gpt-2](https://huggingface.co/transformers/model_doc/gpt2.html) с помощью метода from_pretrained. Так как мы будем использовать её для задачи языкового моделирования, нам потребуется GPT2LMHeadModel, прочитать про неё можно [тут](https://huggingface.co/transformers/model_doc/gpt2.html#gpt2lmheadmodel). У GPT-2 свой токенизатор, про него можно почитать [тут](https://huggingface.co/transformers/model_doc/gpt2.html#gpt2tokenizer).

GPT-2 -- большая модель, время ее работы на CPU будет слишком большим, так что мы сразу отправим её на GPU.

In [50]:
model_name = 'gpt2-medium'
device = 'cuda'

In [6]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

Downloading:   0%|          | 0.00/718 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

In [7]:
model = GPT2LMHeadModel.from_pretrained(model_name)
model = model.to(device)

Downloading:   0%|          | 0.00/1.52G [00:00<?, ?B/s]

### Подсчёт числа параметров модели

Чтобы понять, насколько GPT большая модель, подсчитаем число ее параметров

In [8]:
## Домашнее задание

# TASK: compute number of model parameters
# iterate over model weights and count number of parameters in each of them, then sum it up
# note: you may find model.parameters() method useful
# Our implementation is 2 lines

params = 0
# YOUR CODE STARTS
params = sum([params.numel() for params in model.parameters()])
# YOUR CODE ENDS
params

354823168

Как видим, это большое число параметров, на несколько порядок больше, чем те модели, которые вы обучали до этого! Использовать такое в продакшене как правило невозможно из-за больших требований по ресурсам

# Часть 1. Генерация отзывов моделью GPT

В преыдущем задании мы генерировали ровно 1 токен, так как нам нужно было предсказать сентимент, а оба слова "positive" и "negative" есть в словаре GPT.

В этом задании мы пойдём дальше и самостоятельно реализуем top-k сэмплирование, чтобы генерировать отзывы на фильмы, подобные тем, что в датасете IMDB.

Мы будем продолжать отзывы, используя в качестве "затравки" для модели начало реальных ревью.

### Top-k & Top-p sampling

Чтобы генерация текста была более разнообразной, мы будем использовать top k сэмплирование, которое мы изучали в этом юните. Его смысл в том, что на каждом шаге мы перед сэмплированием зануляем вероятности всех слов, кроме k самых вероятных.

In [9]:
# Домашнее задание

def topk_sample(scores, k):
    """
    Sample from logits using multinomial distribution
    Before sampling, all logits except k greatest are zeroed out.
    Args:
        scores: scores for every word in the vocabulary. Must be 1-dimensional: (num_words_in_vocab)
        k: int, how many hypotheses to sample from
    Returns:
        1 draw from dictribution, torch.LongTensor of shape (1)
    """
    assert k >= 1, 'k must be >=1'
    assert scores.ndim == 1, 'logits must have 1 dimension' 

    # TASK: implement top-k sampling
    # First, get top k values and theoir indices using torch.topk
    # 2nd, fill logits with some small value (e. g. 1e-6)
    # Next, move values back to their place. Note that you will find ellipsis (...) and None useful
    # Finally, use softmax to obtain probabilities and get 
    # Our implementation is 6 lines
    
    # YOUR CODE STARTS

    predicted_ids = scores.clone()
    predicted_ids.fill_(-1e6)

    vals, inds = scores.topk(k) 

    predicted_ids[inds] = vals

    predicted_ids = F.softmax(predicted_ids,dim=-1)

    # YOUR CODE ENDS
    return predicted_ids

Проверка кода:

In [10]:
for _ in range(10):
  logits = torch.randn(5)
  print(f'in:  {logits}')
  print(f'out: {topk_sample(logits, 2)}','\n')

in:  tensor([-1.1256,  1.5893, -1.2233, -0.4460, -1.6299])
out: tensor([0.0000, 0.8845, 0.0000, 0.1155, 0.0000]) 

in:  tensor([-0.3856, -0.0810,  0.0271, -0.3725, -0.7134])
out: tensor([0.0000, 0.4730, 0.5270, 0.0000, 0.0000]) 

in:  tensor([ 2.8173,  0.6406,  1.1968, -1.8260,  0.7936])
out: tensor([0.8349, 0.0000, 0.1651, 0.0000, 0.0000]) 

in:  tensor([-1.2056,  0.1685,  0.4811, -1.3436, -0.5050])
out: tensor([0.0000, 0.4225, 0.5775, 0.0000, 0.0000]) 

in:  tensor([ 0.9245,  0.3863,  0.9873, -1.1035,  0.1816])
out: tensor([0.4843, 0.0000, 0.5157, 0.0000, 0.0000]) 

in:  tensor([ 0.3306,  0.8258,  0.8079, -1.0557, -0.9766])
out: tensor([0.0000, 0.5045, 0.4955, 0.0000, 0.0000]) 

in:  tensor([ 0.8606, -0.3469, -2.1483,  0.6126, -1.4041])
out: tensor([0.5617, 0.0000, 0.0000, 0.4383, 0.0000]) 

in:  tensor([-0.5253,  0.9481,  0.1531,  1.5525, -1.2911])
out: tensor([0.0000, 0.3533, 0.0000, 0.6467, 0.0000]) 

in:  tensor([-0.6507,  0.3133, -0.7402, -0.3925,  1.8440])
out: tensor([0.0000, 

In [11]:
# logits = torch.randn(5)
# for _ in range(10):
#     res1 = topk_sample(logits, 1)
#     res2  = topk_sample(logits, 1)
#     assert res1.ndim == 0  ## <<< Not correct by task condition

#     assert res1.equal(res2)

# not_equal = False
# for _ in range(10):
#     if not topk_sample(logits, 2).equal(topk_sample(logits, 2)): ## <<< controversial condition check
#         not_equal = True
# assert not_equal
# print('OK!')

Также мы будем использовать top-p sampling, который мы уже проходили на лекциях.

In [12]:
# Домашнее задание

def topp_sample(scores, p):
    """
    Sample from logits using multinomial distribution
    Before sampling, all logits except those largest
    whose cumsum exceeds p are zeroed out
    Args:
        scores: scores for every word in the vocabulary. Must be 1-dimensional: (num_words_in_vocab)
        p: float, cumulative probability of most high-scored tokens
    Returns:
        1 draw from dictribution, torch.LongTensor of shape (1)
    """
    assert  0 < p < 1, 'p must be between 0 and 1'
    assert scores.ndim == 1, 'logits must have 1 dimension' 

    # TASK: implement top-p sampling
    # 1. sort scores with descending order
    # 2. get cumulative sum of softmaxed logits
    # 3. get indices when cumulative sum is larger than p. These indices should be zeroed out
    # 4. get real indices that should be zeroes out from sorted indices. Use tensor slicing
    # 5. fill these indices with some small value (e. g. -1e6)
    # 6. sample, as you did in top-k sampling
    # Our implementation is 7 lines
    
    # YOUR CODE STARTS
    
    xval, xind = torch.sort(scores, descending=True) # сортировка входного вектора и сохранение отсортированных значений и их исходных индексов
    
    xval = F.softmax(xval,dim=-1)

    xvalcumsum = torch.cumsum(xval,dim=-1)

    for i in range(xval.shape[0]):    # обнуление всех значений после достижения накопленной вероятности p
      if xvalcumsum[i] > 0.8:
        xval[i+1:] = -1e6
        break

    predicted_ids = xval.clone()
    predicted_ids[xind] = xval # возвращение значений по исходным индексам
    
    # YOUR CODE ENDS
    return F.softmax(predicted_ids,dim=-1)

Проверка кода:

In [13]:
# Часть 
# logits = torch.Tensor([-.1, -.2, .9])   
# for _ in range(10):
#     res1 = topp_sample(logits, 0.8)
#     res2  = topp_sample(logits, 0.8)
#     assert res1.ndim == 0

#     assert res1.equal(res2)
# logits = torch.Tensor([.5, .5, .5])
# not_equal = False
# for _ in range(10):
#     if not topp_sample(logits, 0.8).equal(topp_sample(logits, 0.8)):
#         not_equal = True
# assert not_equal
# print('OK!')

In [14]:
# z = torch.tensor([-0.4105, -1.4342,  2.6922, -0.1102, -0.2479])
z = torch.Tensor([-.1, -.2, .9])

print(f'in:\t {z}')
print(f'smax:\t {F.softmax(z,dim=0)}')
print(f'out:\t {topp_sample(z, 0.8)}')

in:	 tensor([-0.1000, -0.2000,  0.9000])
smax:	 tensor([0.2163, 0.1957, 0.5880])
out:	 tensor([0.4081, 0.0000, 0.5919])


In [15]:
for _ in range(10):
  logits = torch.randn(5)
  print(f'in:  {logits}')
  print(f'out: {topp_sample(logits, 0.8)}','\n')

in:  tensor([ 1.0016,  0.0354, -0.9133, -1.8887, -0.3428])
out: tensor([0.4194, 0.2997, 0.0000, 0.0000, 0.2809]) 

in:  tensor([ 0.5689,  0.7551, -0.5713,  0.4813,  0.4783])
out: tensor([0.2494, 0.2614, 0.0000, 0.2447, 0.2445]) 

in:  tensor([ 0.8534,  0.0608,  1.0421, -0.0794, -0.3659])
out: tensor([0.2668, 0.2266, 0.2839, 0.2226, 0.0000]) 

in:  tensor([-0.5800,  1.8308, -2.1892, -0.5793, -0.8938])
out: tensor([0.0000, 0.6727, 0.0000, 0.3273, 0.0000]) 

in:  tensor([0.1809, 0.3830, 0.2818, 2.2337, 0.4611])
out: tensor([0.0000, 0.2701, 0.0000, 0.4577, 0.2722]) 

in:  tensor([ 0.5310,  0.5479,  0.9969, -0.7187,  0.5424])
out: tensor([0.2419, 0.2427, 0.2730, 0.0000, 0.2424]) 

in:  tensor([ 0.9272,  1.3809,  0.6884,  0.2582, -1.5839])
out: tensor([0.3222, 0.3725, 0.3053, 0.0000, 0.0000]) 

in:  tensor([-1.6031, -0.9918, -0.6458, -0.2329, -1.1511])
out: tensor([0.0000, 0.2348, 0.2518, 0.2843, 0.2291]) 

in:  tensor([ 0.2995, -0.1212,  0.1805, -0.0863, -0.1727])
out: tensor([0.2627, 0.240

Теперь напишите функцию генерации текста:

In [16]:
def generate_text(model, ids, length, k=None, p=None):
    """
    Generate text with language model with ids as prompt
    Args:
        model: huggingface LM model
        ids: input ids, from tokenizer.encode, must be 2-dimensional
        length: int, how many tokens to geenrate
        k: int, how many hypotheses to sample from in top-k sampling
    Returns:
        token ids from generated text, together with token ids of prompts, 2d LongTensor
    """
    assert length >= 1
    if k and p:
        raise RuntimeError('Cannot use topk and topp sampling simultaneously')
    # TASK: write generation loop
    
    # YOUR CODE STARTS
    ids = model.generate(
        ids,
        max_length=length,
        repetition_penalty=1.2,
        do_sample=True,
        top_k=k,
        top_p=p
        )

    ids.squeeze_()
    # YOUR CODE ENDS
    return ids.cpu().detach()


Проверка кода:

In [17]:
# Assert is not correct

# ids = torch.randint(0, 1000, (4,)).to(device)
# ids.unsqueeze_(dim=0)
# generation = generate_text(model, ids, 4, k=2)

# assert generation.shape == torch.Size([5])

# generation = generate_text(model, ids, 3, p=.5)

# assert generation.shape == torch.Size([5])

### Запускаем генерацию

Зададим модели какой-нибудь prompt, например восторженный отзыв о фильме, и посмотрим, как она продолжит его. Так как модель сэмплит на каждом шаге, нам интересно посмотреть на несколько траекторий сразу. Для этого мы и писали батч-режим в генерации.

In [18]:
prompt = 'What a lovely film !'

Подготовим входные данные для GPT

In [19]:
ids = tokenizer.encode(prompt, return_tensors="pt")

Запускаем генерацию! Можно запустить на 10-20 токенов, можно и больше

In [20]:
n_tries = 5

In [21]:
generations = [generate_text(model, ids, length=70, k=10).cpu().detach() for _ in range(n_tries)]

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


In [22]:
for generation in generations:
    print(tokenizer.decode(generation))
    print('\n' + '=' * 40 + '\n')

What a lovely film! I love the colour scheme as well. The lighting is perfect, and it's just so funny :). It has some fantastic moments of humour too:

I've watched this several times now (and still can't get over how amazing every frame looks!)<|endoftext|>


What a lovely film! It is about an old girl with cancer and the boy who tries to help her but it's really sad. But what makes my eyes water in particular :D

The only thing I don't like: some actors were too young, others seemed pretty childish or just plain ugly for their age (eagle). So they


What a lovely film!
This was not the first time I have seen this one. It is my last viewing of it and i am really glad that they did so well with it, because in fact you could say that their films always make me laugh or smile :) In all honesty if you like comedy movies (or love them) please watch THIS


What a lovely film! It is one of my favourite films from 2001 and I'm still waiting for the sequel.

I would like to thank you all again

А теперь запустим генерацию с top-p сэмплированием

In [23]:
generations = [generate_text(model, ids, length=70, p=0.5).cpu().detach() for _ in range(n_tries)]
for generation in generations:
    print(tokenizer.decode(generation))
    print('\n' + '=' * 40 + '\n')

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


What a lovely film! I have to admit that the ending was very surprising. It is one of those films which makes you wonder how they did it without any real plot or character development, but at least with all these characters and story lines in place.


The soundtrack has been great too - so much more varied than anything else out there!


What a lovely film!
I really enjoyed the opening scene of this movie. It was so nice to see that I could be in my own home and not have any fear or anxiety, just relax with all these beautiful trees around me (or as they call them here: "tree-people" ).  The ending is pretty much what you'd


What a lovely film!
The story is set in the early days of an American colony, and it's quite interesting to see how many things are different today. For example: The US military has now expanded its control over Africa with their own drones that can carry out assassinations; they have also created "bomber" aircraft which kill people


What a lovely film!


I was g

# Часть 2. Few-shot learning

Одной из особенностей GPT, про которую авторы написали, является способность к few-shot learning. В этой части домашнего задания мы на примере хорошо знакомой нам задачи классификации текстов исследуем, как хорошо модель умеет различать сентимент отзывов без дополнительного обучения.

## Датасет.
Мы будем снова использовать датасет imdb, на котором мы уже обучили несколько моделей. Так как обучать саму модель мы не будем (в few-shot парадигме веса не меняются), то можем сразу взять валидационную часть датасета.

In [24]:
text_dataset = datasets.load_dataset("imdb")
texts = text_dataset["test"]["text"]
labels = text_dataset["test"]["label"]

Downloading:   0%|          | 0.00/1.92k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/84.1M [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset imdb downloaded and prepared to /root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/e3c66f1788a67a89c7058d97ff62b6c30531e05b549de56d3ab91891f0561f9a. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

### Метки для задачи классификации на естественном языке
Поскольку языковая модель знает только токены языка и не знает маппинга между индексами и лейблами, нам нужно будет сравнивать лейблы напрямую. С одной стороны, это создаёт дополнительные сложности: модель может предсказать "Positive Sentiment" вместо "positive", поэтому нам потребуется минимальный постпроцессинг, чтобы учесть большинство таких случаев.

In [25]:
mapping = {idx: label for idx, label in enumerate(text_dataset["train"].features["label"].names)}
mapping

{0: 'neg', 1: 'pos'}

### Игрушечный датасет
Чтобы быстрее прототипировать наши вводы (prompts), полезно взять маленькую часть датасета и проверять гипотезы на ней. Так как imdb упорядочен по лейблам, а самих лейблов всего 2, мы берём одинаковое количество примеров с начала и с конца.

In [26]:
def get_toy_dataset(dataset, split, length = 100):
    texts = dataset[split]["text"]
    labels = dataset[split]["label"]
    small_texts = texts[:length // 2] + texts[-length//2:]
    small_labels = labels[:length // 2] + labels[-length//2:]
    return small_texts, small_labels

In [27]:
assert len(get_toy_dataset(text_dataset, "test", 100)[0]) == 100

## Few-shot learning

В данном задании мы будем не обучать модель на данных, а использовать уже обученную на огромном корпусе языковую модель, чтобы "угадывать" сентимент отзывов.

Модель будет видеть в качестве затравки (prompt) описание задачи на естественном языке.

Нам потребуется:
- описание задачи
- лейблы
- несколько (хотя бы 3) примеров
- вспомогательные маркеры (разделитель примеров и маркер конца входа)

Например:

```
Translate from English to French
sea otter => loutre de mer
peppermint => menthe poivrée
plush girafe  => 
```

Первая строка здесь -- описание задачи. Мы решаем задачу классификации, поэтому нам нужно
описать задачу примерно как `To which category does the text belong? positive , negative `

Вторая и третья строки -- примеры входа и выхода. В нашем случае примерами входа и выхода будут служить тексты ревью и метки классов positive и negative (а не 0 и 1, как раньше).

Наконец, последняя строка -- это тот текст, который модель должна классифицировать. Модель уже "видела" выше пример того, что после маркера конца входа идёт сентимент, и мы ожидаем, что она сможет выучить эту закономерность.

В данном задании часть входа будет написана за вас, вам надо будет придумать обучающие примеры для few-shot learning'а

In [28]:
marker = ' => '
separator = '\n'

In [29]:
labels_tmp = ["positive" , "negative"]
labels_text = " , ".join(labels_tmp)
task_description = f'To which category does the text belong?: {labels_text} ' + separator

In [30]:
examples = [
    # TASK define examples
    # They should be a tuple with text and label
    # label should be from "labels"
    # Get 3-10 different examples, they should cover both positive 
    # and negative reviews
    # YOUR CODE STARTS
    ["resolution is an interesting one","positive"],
    ["I liked this film","positive"],
    ["voters see the brilliance in this script","positive"],
    ["the worst thing I have ever seen","negative"],
    ["This is clearly the worst","negative"],
    ["Terrible. Horrible acting.","negative"]
    # YOUR CODE ENDS
]

In [31]:
assert len(examples) > 1, 'Write at least two different examples'
for ex in examples:
    assert ex[1] in labels_tmp

In [32]:
input = task_description  + separator.join([marker.join(example) for example in examples])

In [33]:
print(input)

To which category does the text belong?: positive , negative 
resolution is an interesting one => positive
I liked this film => positive
voters see the brilliance in this script => positive
the worst thing I have ever seen => negative
This is clearly the worst => negative
Terrible. Horrible acting. => negative


Сравните получившийся инпут с примером выше. Мы получили то, что надо!

In [34]:
n_chars = 50

In [35]:
def get_prompt(beginning, separator, text, marker) -> str:
    return ' '.join([input,  separator, text[:n_chars] + text[-n_chars:], marker])

## Время проверить работу нашей модели на маленьком датасете!

In [36]:
small_texts, small_labels = get_toy_dataset(text_dataset, "test")

Так как мы предсказываем сентимент текста, каждый из который задаётся одним токеном, то нам
- не надо сэмплировать. Мы хотим, чтобы модель честно учитывала вероятности для токенов
- достаточно сделать один шаг генерации. Модель должна будет предсказать лейбл текста, а он, как мы выяснили, однотокенный.

Таким образом, нам просто надо сделать один шаг жадного предсказания (greedy decoding).

In [37]:
# Домашнее задание

def predict_on_text(model, ids) -> int:
    """
    Run Language Model on text and use logits from last time 
    step to predict sentence category with greedy decoding
    Args:
        model: huggingface LM model
        ids: LongTensor . Text encoded with tokenizer.
    Returns:
        model prediction, index of most probable word
    """
    # TASK: run a model and get index most probable logit
    # HINT: do not forget to detach tensors
    # HINT 2: GPT-2 model returns tuple where logits are stored in the
    # 1st item. You do not need 2nd item at all
    # Our implementation is 2 lines
    # YOUR CODE STARTS
    idx = model.generate(
        ids.reshape(-1, ids.shape[0]),
        max_length=ids.shape[0] + 1,
        pad_token_id=tokenizer.eos_token_id,
    )[0][-1]
    # YOUR CODE ENDS
    return idx.cpu().detach().item()

Проверка кода:

In [38]:
ids = tokenizer.encode('hello world !', return_tensors="pt",)
prediction = predict_on_text(model, ids.to(device))
assert isinstance(prediction, int)

In [39]:
def predict_on_dataset(model, texts, target, mapping):
    """
    Run Language Model on a dataset and use its predictions as
    labels for sentences
    Args:
        model: huggingface LM model
        texts: List[str], texts of dataset
        target: List[int], indices of classes
        mapping: Dict[int, str] mapping from class indices to class labels
    Returns:
        model predictions, "as is", List[str]
        target labels, after mapping, List[str]
    """
    if len(texts) != len(target):
        raise RuntimeError('Texts and target lengths mismatch')
    # max_length has additional -1 because we will generate exactly 1 token 
    # with LM
    max_length = model.config.n_ctx - 1
    predictions = []
    labels = []
    for idx in tqdm(range(len(texts))):
        text, label = texts[idx], target[idx]
        ids = tokenizer.encode(
            get_prompt(input, separator, text, marker), 
            add_special_tokens=False, 
            return_tensors="pt",
            max_length=max_length
        ).squeeze()
        idx = predict_on_text(model, ids.to(device))
        predictions.append(tokenizer.decode([idx]))  # generation.cpu().numpy()[0][-1]
        labels.append(mapping[label]) 
    return predictions, labels

In [40]:
predictions, target = predict_on_dataset(model, small_texts, small_labels, mapping)

100%|██████████| 100/100 [02:51<00:00,  1.72s/it]


### Замер качества

Посмотрим, что выдаёт нам языковая модель:

In [41]:
print(predictions[:10])

[' negative', ' negative', ' negative', ' negative', ' negative', ' positive', ' negative', ' positive', ' positive', ' negative']


Можно заметить, что предсказанные метки не совсем похожи на pos и neg, которые были у нас в таргете. Напишем простую функцию, которая нормализует предсказания модели

In [42]:
# Домашнее задание

def normalize_prediction(raw_output: str) -> str:
    """
    Helper that transforms model prediction to normal
    For example, ' positive' -> 'pos', ' Negative' -> neg
    Args:
        raw_output: str, output from language model
    Returns:
        normalized value: no leading/trailing spaces, lowercase,
        at most 3 chars long
    """
    # TASK: write prediction normalizer
    # You need to remove spaces, lowercase and get only 3 leading chars
    # Our implementation is 1 line
    # YOUR CODE STARTS
    normalized_output = [x.strip().lower()[:3] for x in raw_output]
    # YOUR CODE ENDS
    return normalized_output

Напишем функцию, вычисляющую точность:

In [43]:
def accuracy(predictions, labels) -> float:
    """
    Helper that transforms model prediction to normal
    For example, ' positive' -> 'pos', ' Negative' -> neg
    Args:
        predictions: List[str], model predictions, should be labels in natural language
        labels: List[str], target labels
    Returns:
        accuracy, float from [0, 1]
    """
    if not labels:
        # sanity check to avoid zero division
        return 0
    if len(predictions) != len(labels):
        raise ValueError(f'Predictions and labels have mismatched length')
    total = len(predictions)
    match = 0
    # TASK: calculate accuracy
    # For every pair of predicted and real labels, check that they coincide.
    # You can use zip() method for iteration over two sequences
    # simultaneously.
    # Our implementation is 3 lines
    # YOUR CODE STARTS
    for i,j in zip(predictions, labels):
      if i==j:
        match+=1
    accuracy = match/total
    # YOUR CODE ENDS
    return accuracy

In [47]:
normalized_predictions = normalize_prediction(predictions)

In [48]:
accuracy(normalized_predictions, target)

0.68

### Ура, модель без обучения работает лучше случайного угадывания!

Тем не менее, это заметно хуже, чем у полносвязной нейросети. **Учитывая требования по ресурсам и время 
работы, использование такой модели в реальных задачах непрактично!**

## А теперь замер на всем тестовом сете:

In [55]:
predictions, target = predict_on_dataset(model, texts, labels, mapping)
normalized_predictions = normalize_prediction(predictions)
accuracy(normalized_predictions, target)

100%|██████████| 25000/25000 [33:46<00:00, 12.34it/s]


0.65248

Все задание считается выполненным, если вы достигли accuracy 0.6 и выше на тестовом датасете. 

Данный пример призван продемонстрировать, что большая языковая модель хоть и справляется с задачей без лишних данных, но делает это заметно хуже специально созданных для задачи моделей, к тому же работает непростительно долго.

### Резюме

В этом задании мы применили большие языковые модели на практике и решили с её помощью 2 задачи:
- conditional генерация текста
- few-shot классификация текстов

Мы увидели, что модель генерирует связный текст и может решать задачу классификации без изменений своих весов.

Несмотря на возможности языковых моделей, их применение на практике всё ещё осложнено большими потребляемыми ресурсами, а также невозможностью интерпретировать предсказания.